In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [3]:
data = pd.read_csv("..\properatti.csv", index_col=0)
data.shape

(121220, 25)

In [3]:
# setea pandas para no truncar los valores de las columnas
pd.set_option('display.max_colwidth', None)

In [4]:
# defino dos grupos de columas para poder trabajar con ellas de forma mas sencilla
places = ['place_name','place_with_parent_names', 'country_name','state_name']
geolocation = ['geonames_id', 'lat-lon','lat','lon']

# creo un nuevo dataframe con las columnas que me interesan
location_data = data[places + geolocation].copy()
location_data.head()

,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon
0,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839
1,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330
2,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982
3,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424
4,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447


Exploracion de las columnas relacionadas con la ubicacion: Por Provincia, Ciudad/Barrio, el conjunto de ubicación 
- state_name 
- place_name                        
- place_with_parent_names              
vamos a:
- Explorar las columnas 
- Visualizar los nan de estas columnas
- Corroborar que hay relacion entre el ciudad/barrio o "place_name" y la concatenacion de ubicacion en "place_with_parent_names" para ver si efectivamente corresponden o hay barrios mezclados

- Exploramos las columnas 

In [10]:
data.loc[:, ["place_with_parent_names", 'state_name', 'place_name']].sample(10)

,place_with_parent_names,state_name,place_name
69580,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Bs.As. G.B.A. Zona Sur,Turdera
113427,|Argentina|Córdoba|Córdoba|,Córdoba,Córdoba
89342,|Argentina|Santa Fe|Rosario|,Santa Fe,Rosario
53890,|Argentina|Santa Fe|Rosario|,Santa Fe,Rosario
83961,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Bs.As. G.B.A. Zona Norte,Olivos
76545,|Argentina|Bs.As. G.B.A. Zona Sur|Almirante Br...,Bs.As. G.B.A. Zona Sur,Burzaco
68492,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Bs.As. G.B.A. Zona Sur,Temperley
105706,|Argentina|Córdoba|,Córdoba,Córdoba
66867,|Argentina|Córdoba|,Córdoba,Córdoba
96537,|Argentina|Capital Federal|Liniers|,Capital Federal,Liniers


In [4]:
#count: cuenta el numero de registros (no incluye null)
#unique:  número de objetos distintos en la columna,
#top: es el dato más frecuente que se produce,
#freq: es la cantidad de veces que aparece el objeto “top” en la columna

data[["place_with_parent_names", "state_name", "place_name"]].describe()

,place_with_parent_names,state_name,place_name
count,121220,121220,121197
unique,1164,28,1060
top,|Argentina|Santa Fe|Rosario|,Capital Federal,Córdoba
freq,8504,32316,9254


In [5]:
#ubicacion por provincia "State_name"
#notamos que los datos estan concentrados en Buenos Aires (Cap fed, zona norte, zona sur, oeste, atlantica, interior),
#  Cordoba, Santa fe 

data["state_name"].value_counts() #Excludes NA values by default.

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
Santiago Del

In [6]:
#ubicacion por ciudad-partido o barrio
 
data["place_name"].value_counts().head(20)

#Tigre es una ciudad al norte de la ciudad de Buenos Aires.
# Nordelta es una localidad urbana en el Partido de Tigre, Provincia de Buenos Aires 

##esta columna mezcla ciudades y barrios

#Capital Federal está como place_name, 

Córdoba            9254
Rosario            8504
Mar del Plata      6534
Tigre              3324
Nordelta           3315
Belgrano           2992
Palermo            2885
La Plata           2534
Caballito          2273
Pilar              2228
Villa Urquiza      1632
Olivos             1590
Recoleta           1547
Lomas de Zamora    1529
Flores             1354
Ituzaingó          1340
Villa Crespo       1331
Capital Federal    1297
San Telmo          1216
Morón              1209
Name: place_name, dtype: int64

In [5]:
#Vemos algunos registros de place_with_pa..
data["place_with_parent_names"].value_counts().head(10) 

#el atributo reperesenta el pais/ "country_name", la provincia/ "state_name" (o division de la provincia), la ciudad principal/partido-municipio o barrio en el caso de capital federal"place_name"
#y un adicional que puede ser una localidad del partido (ejem: |Argentina|Bs.As. G.B.A. Zona Norte|Tigre|*Nordelta*|)
#el registro tiene entre 2  y 4 concatenaciones: 2 concantenaciones solo muestra hasta la ubicacion por provincia, no brinda informacion relevante


###|Argentina|Capital Federal|  1297 no tiene nombre de barrio, desestimar o completar? 
#|Argentina|Córdoba|    2648

#esta columna puede usarse para completar los datos faltantes de place_name 
#podemos chequear si el atributo "place_name" coincide con los datos aquí plasmados

|Argentina|Santa Fe|Rosario|                              8504
|Argentina|Córdoba|Córdoba|                               6606
|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|    6534
|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nordelta|       3315
|Argentina|Capital Federal|Belgrano|                      2992
|Argentina|Capital Federal|Palermo|                       2885
|Argentina|Córdoba|                                       2648
|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|                2359
|Argentina|Capital Federal|Caballito|                     2273
|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|                1857
Name: place_with_parent_names, dtype: int64

In [4]:
#Para state_name vemos: cuales son los datos de place_name más frecuente, su frecuencia y cuantos refistros diferentes existen 

data.groupby(["state_name"])["place_name"].describe()

#el traibuto de place_name está representado principalmente por una ciudad-partido de la provincia

#count: cuenta el numero de registros no incluye null
#unique:  número de objetos distintos en la columna,
#top: es el dato más frecuente que se produce,
#freq: es la cantidad de veces que aparece el objeto “top” en la columna

,count,unique,top,freq
state_name,,,,
Bs.As. G.B.A. Zona Norte,25537,264,Tigre,3324
Bs.As. G.B.A. Zona Oeste,9322,78,Morón,1209
Bs.As. G.B.A. Zona Sur,13952,123,La Plata,2534
Buenos Aires Costa Atlántica,10006,87,Mar del Plata,6534
Buenos Aires Interior,2291,99,Bahía Blanca,480
Capital Federal,32316,62,Belgrano,2992
Catamarca,27,6,Manantiales,17
Chaco,57,7,Resistencia,37
Chubut,259,16,Cushamen,100


- Visualizar los nan de las columnas
de aplicar la funcion isna().sum() en data obtuvimos
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0

#solo la columna place_name tiene valores nulos

In [12]:
#vemos que filas son y como podemos completarlas o no

data[data.place_name.isna()].head(2)

#los 23 datos pueden completarse con fillna con el dato de "Tigre" ya que vimos en place_name que "Tigre" 3324, un valor signif
#si escogemos como representativa la ciudad, sino buscamos el barrio? 

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
6489,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4008968545,-58.63809847",-34.400897,-58.638098,...,240.0,2708.333333,2708.333333,NaN,NaN,NaN,http://www.properati.com.ar/15ui7_venta_casa_t...,"Espectacular casa moderna de una planta, con a...",Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/7/vQjgbHSz1fKXxy...
10201,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.400704,-58.638067",-34.400704,-58.638067,...,300.0,NaN,1783.333333,NaN,5.0,NaN,http://www.properati.com.ar/166p5_venta_casa_t...,Corredor Responsable: SABRINA COCCONI - CSI 62...,VENTA CASA SANTA MARÍA DE TIGRE COUNTRY BARRIO...,https://thumbs4.properati.com/1/FABpKIJMaoAO3t...


#### Corroborar si place_with_parent_names coincide con country_name, state_name, place_name 

In [5]:
# convierte la columna plase_with_parent_names en una lista usando el separador "|"
location_data['place_with_parent_names'] = location_data['place_with_parent_names'].apply(lambda x: x.lstrip("|").rstrip("|").split("|"))
location_data['place_with_parent_names'].head()

0                             [Argentina, Capital Federal, Mataderos]
1                       [Argentina, Bs.As. G.B.A. Zona Sur, La Plata]
2                             [Argentina, Capital Federal, Mataderos]
3                               [Argentina, Capital Federal, Liniers]
4    [Argentina, Buenos Aires Costa Atlántica, Mar del Plata, Centro]
Name: place_with_parent_names, dtype: object

In [6]:
# funcion para chequear si los valores de la columna place_with_parent_names coinciden con 
# los valores de las columnas country_name, state_name, place_name

def is_location_different(row):
    # la lista debería tener 3 elementos
    if len(row['place_with_parent_names']) != 3:
        return True
    if row['country_name'] == row['place_with_parent_names'][0] \
    and row['state_name'] == row['place_with_parent_names'][1] \
    and row['place_name'] == row['place_with_parent_names'][2]:
        return False
    else:
        return True

In [7]:
# crea la mascara utilizando la función is_location_different
mask = location_data.apply(lambda x: is_location_different(x), axis=1)
print("place_with_parent_names difference with country_name, state_name and place_name:", location_data[mask].shape[0])
location_data[mask][places].head()

place_with_parent_names difference with country_name, state_name and place_name: 45220


,place_name,place_with_parent_names,country_name,state_name
4,Centro,"[Argentina, Buenos Aires Costa Atlántica, Mar del Plata, Centro]",Argentina,Buenos Aires Costa Atlántica
6,Munro,"[Argentina, Bs.As. G.B.A. Zona Norte, Vicente López, Munro]",Argentina,Bs.As. G.B.A. Zona Norte
12,Martínez,"[Argentina, Bs.As. G.B.A. Zona Norte, San Isidro, Martínez]",Argentina,Bs.As. G.B.A. Zona Norte
13,Palermo Soho,"[Argentina, Capital Federal, Palermo, Palermo Soho]",Argentina,Capital Federal
14,Palermo Soho,"[Argentina, Capital Federal, Palermo, Palermo Soho]",Argentina,Capital Federal


Existen 45220 registros que no coinciden exactamente. 

In [8]:
# cuenta la cantidad de valores de cada fila de la lista place_with_parent_names
location_data['place_with_parent_names'].apply(lambda x: len(x)).value_counts()

3    76023
4    39869
2     4780
5      548
Name: place_with_parent_names, dtype: int64

Los que no coinciden exactamente son aquellas lsita que no tienen 3 valores. Primero revisamos que los valores que están en cada lista tengan su contraparte en las otras columnas 

In [9]:
# country_name
print("null values:", location_data['country_name'].isna().sum())
mask = ~location_data.apply(lambda x: x['country_name'] in x['place_with_parent_names'], axis=1)
print("place_with_parent_names difference with country_name:", location_data[mask].shape[0])

null values: 0
place_with_parent_names difference with country_name: 0


In [10]:
# state_name
print("null values:", location_data['state_name'].isna().sum())
mask = ~location_data.apply(lambda x: x['state_name'] in x['place_with_parent_names'], axis=1)
print("place_with_parent_names difference with state_name:", location_data[mask].shape[0])

null values: 0
place_with_parent_names difference with state_name: 0


In [11]:
# place_name
print("null values:", location_data['place_name'].isna().sum())

mask = ~location_data.apply(lambda x: x['place_name'] in x['place_with_parent_names'], axis=1)
location_data[mask].head()



null values: 23


,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon
6489,NaN,"[Argentina, Bs.As. G.B.A. Zona Norte, Tigre]",Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4008968545,-58.63809847",-34.400897,-58.638098
10201,NaN,"[Argentina, Bs.As. G.B.A. Zona Norte, Tigre]",Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.400704,-58.638067",-34.400704,-58.638067
11451,NaN,"[Argentina, Bs.As. G.B.A. Zona Norte, Tigre]",Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026880384,-58.6262613379",-34.402688,-58.626261
14839,NaN,"[Argentina, Bs.As. G.B.A. Zona Norte, Tigre]",Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4007994779,-58.6381735719",-34.400799,-58.638174
18622,NaN,"[Argentina, Bs.As. G.B.A. Zona Norte, Tigre]",Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4032781195,-58.6263503945",-34.403278,-58.626350


Los registros que no tienen valor en place_name son los que no coinciden, todos del municipio de Tigre.  

# TODO: Resta ver y analizar los que tienen más de 3 valores en la lista.

Otra forma de ver si los datos de place_with_parent_names son los de place_name